# Project 8 - Pricing Fixed Income Securities

In [244]:
# python set up
import matplotlib.pyplot as plt
import math
from numpy import*
import pandas as pd
from scipy.stats import * 
from scipy.optimize import newton
set_printoptions(threshold=float('inf'), linewidth= 200, suppress = True)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
random.seed(7)

## Question 1

Assume the dynamics of the short-term interest rate, under the risk-neutral measure, are given by
the following SDE (Vasicek model) : 

$$dr_t =\kappa (\bar{r}− r_{t})dt + \sigma dW_t$$

with $r_0 = 5$%, $\sigma$ = 10%, $\kappa$ = 0.82, $\bar{r}$= 5%

**(a)** Use Monte Carlo Simulation (assume each time step is a day) to find the price of a pure discount
bond, with Face Value of \$1,000, maturing in T = 0.5 years (at time 𝑡 = 0):

$$P(t, T) = \ E\ [ \ $1,000\  exp (− \int_t^T r(s) ds)]$$

**Solution:**

Simulate 10,000 paths of interest rates by Vasicek Model

In [245]:
def Vasicek(r0, T, paths, steps):
    
    sd = 0.1 
    kappa = 0.82
    r_mean= 0.05
    
    dt = T/steps 
    rt = zeros((paths, steps + 1))
    rt[:, 0] = r0
    for i in range(1, steps + 1):
        rt[:,i] = rt[:,i-1] + kappa*(r_mean - rt[:,i-1]) *dt + sd* sqrt(dt)*random.normal(0, 1, paths)
    return rt

Price the zero coupon bond using the short rate paths

In [246]:
def zeroCouponBondVasicek(r0, t, T, F):
    
    # intialize parameters
    r0= 0.05 
    sd = 0.1 
    kappa = 0.82
    r_mean= 0.05
    
    paths = 1000
    steps = int(365*(T - t))
    dt = (T - t)/steps
    r = Vasicek(r0, (T - t), paths, steps)
    
    P = mean(F * exp(-sum(r, axis = 1)*dt))
    
    return P

In [247]:
print("The price of the zero coupon bond under Vasicek: $", 
      round(zeroCouponBondVasicek(r0 = 0.05, t = 0, T = 0.5, F = 1000), 5))

The price of the zero coupon bond under Vasicek: $ 975.75607


**(b)** Use Monte Carlo Simulation to find the price of a coupon paying bond, with Face Value of \$1,000, paying semiannual coupons of $30, maturing in T = 4 years:

$$P(t, T) = \ E\ [ \sum_{i = 1}^8 C_i \  exp (− \int_t^T r(s) ds)]$$


where C = {$C_i$ = \$30 for 𝑖 = 1,2, … ,7; and $C_8$ = \$1,030},

T = {$T_1, T_2, T_3, T_4, T_5, T_6, T_7, T_8$} = {0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4}

**Solution:**

Initialize parameters

In [248]:
coupons = [30] * 7 + [1030]
payment_date = arange(0.5, 4.5, 0.5)

Using the function in (a), the price of the coupon bond could be found 

In [249]:
def couponBondVasicek(r0, t, coupons, payment_date):
    
    # intialize parameters
    r0= 0.05 
    sd = 0.1 
    kappa = 0.82
    r_mean= 0.05
    F = 1000
    
    zero_coupon_bonds = list(map(lambda coupon, coupon_date: zeroCouponBondVasicek(r0, t, T = coupon_date, F = coupon), 
                                 coupons, payment_date))
    P = sum(zero_coupon_bonds)
    
    return P

In [250]:
print("The price of the coupon paying bond under Vasicek : $", couponBondVasicek(r0 = 0.05, t = 0, 
                                                                                 coupons = coupons, 
                                                                                 payment_date = payment_date))

The price of the coupon paying bond under Vasicek : $ 1052.59464595


**(c)** Use Monte Carlo Simulation to find the price of a European Call option on the pure discountbond in part (a). The option matures in 3 months and has a strike price of K = \$980. Use the explicit formula for the underlying bond price (only for the bond price).

Explicit Formula for bond price

$$  P(t, T) =  A(t, T) \large e^{-B(t, T) r_t}$$

Where,

$$ B(t, T) = \large \frac{1}{\kappa}\big(1 - e^{-\kappa(T - t)}\big)$$

 $$ A(t, T) = \large e^{\big(\bar{r} - \frac{\sigma^2}{2\kappa^2}\big)\big(B(t, T) - (T - t)\big) - \frac{\sigma^2}{4\kappa} B^2(t, T)}$$

**Important:** The call option is written on the expectation of the bond price. Therefore, the underlying asset price of this call option is the bond price at the time of option maturity

**Solution:**

Build function that calcualtes the zero-coupon bond explicitly with formulas

In [251]:
def ZeroCouponBondExplicit(rt, t, T, F):
    
    # intialize parameters
    sd = 0.1 
    kappa = 0.82
    r_mean= 0.05
    
    # Find bond price given time t
    B = 1/kappa * (1 - exp(-kappa * (T - t)))
    A = exp(((r_mean - sd**2/(2*kappa**2)) * (B - (T - t)) - sd**2/(4*kappa)*B**2))
    
    return F * A*exp(-B*rt)

Build function that finds the 3-month European Option on the 6-month zero coupon bond

In [252]:
def callOptionOnZeroExplicit(t, T, K, F):
    
    # intialize parameters
    r0= 0.05 
    sd = 0.1 
    kappa = 0.82
    r_mean= 0.05
    
    paths = 15000
    steps = int(365*(T - t))
    dt = (T - t)/steps
    rt = Vasicek(r0, t, paths, steps)
    
    # Find the bond price
    P_bond = ZeroCouponBondExplicit(rt[:,-1], t, T, F)
    
    # Find option payoff
    payoff = maximum(P_bond - K, 0)
    P_option = mean(payoff * exp(-sum(rt, axis = 1)*dt))
    
    return P_option

In [253]:
print("The price of European Option on zero coupon bond (with explicit formula on zero coupon bond): $", 
      round(callOptionOnZeroExplicit(t = 0.25, T = 0.5, K = 980, F = 1000), 5))

The price of European Option on zero coupon bond (with explicit formula on zero coupon bond): $ 8.94896


**(d)** Use Monte Carlo Simulation to find the price of a European Call option on the coupon paying bond in part (b). The option matures in 3 months and has a strike price of K = \$980. Use Monte Carlo simulation for pricing the underlying bond.

**Solution:**

In [254]:
random.seed(7)

In [255]:
def callOptionOnCouponSimulate(t, K, F, coupons, payment_date):
    
    # intialize parameters, t is option maturity
    r0= 0.05 
    sd = 0.1 
    kappa = 0.82
    r_mean= 0.05
    
    paths = 1000
    steps = int(365*(t))
    dt = t/steps
    rt = Vasicek(r0, t, paths, steps)
    
    # Find the bond price
    P_bond = array(list(map(lambda rt: couponBondVasicek(rt, t, coupons, payment_date), rt[:, -1])))
    #P_bond = array([couponBondVasicek(r, coupons, payment_date, t) for r in rt[:, -1]])
    
    # Find option payoff
    payoff = maximum(P_bond - K, 0)
    P_option = mean(payoff * exp(-sum(rt, axis = 1)*dt))
    
    return P_option

In [256]:
#print("The price of European Option on coupon paying bond (with monte carlo simulation): $", 
   #  round(callOptionOnCouponSimulate(t = 0.25, K = 980, F = 1000, coupons = coupons, payment_date = payment_date), 5))

The price of European Option on coupon paying bond under Vasicek (with monte carlo simulation): $ 82.22869

**(e)** Find the price of a European Call option of part (d) by using the explicit formula for the
underlying bond price, and reconcile the findings with the ones of part (d).

**Solution:**

Initialize parameters

In [257]:
coupons = array([30] * 7 + [1030])
payment_date = arange(0.5, 4.5, 0.5)
maturity = 0.25

Build function that finds $\Pi(T, T_i, r^*)$

In [258]:
def PI(rt, maturity, payment_date):
    
    # intialize parameters, maturity is option maturity, payment_date is coupon payment date
    r0= 0.05 
    sd = 0.1 
    kappa = 0.82
    r_mean= 0.05    
    
    # find A(t, T) and B(t, T) according to formula
    B = (1/kappa) * (1 - exp(-kappa * (payment_date - maturity)))
    A = exp((r_mean - (sd**2)/(2*kappa**2)) * (B - (payment_date - maturity) ) - (sd**2)/(4*kappa) * B**2 )
    
    # find pi
    PI_val = A*exp(-B*rt) 
    
    return PI_val

Find optimize function

In [259]:
def f_opt(rt, maturity, payment_date, coupons, K):
    
    # find PI
    PIs = PI (rt, maturity, payment_date)
    value = sum(coupons * PIs) - K
    
    return value

Build function that calculates the zero coupon bond price explicitly

In [260]:
def callOptionOnCouponBS( maturity, K, F, coupons, payment_date):
    
    # intialize parameters, t is option maturity
    r0= 0.05 
    sd = 0.1 
    kappa = 0.82
    r_mean= 0.05    
    
    # find r*
    r_ = newton(f_opt, 0, args=( maturity, payment_date, coupons, K))

    #find Ki
    Ki = PI(r_, maturity, payment_date)
    
    # find P(t, Ti)
    P_t_Ti = PI(r0, 0, payment_date)
    P_t_T = PI(r0, 0, maturity)
    
    # find d1, d2
    sd_p = sd*( (1 - exp(-kappa * (payment_date - maturity)) ) / kappa) * sqrt((1 - exp(-2*kappa*(maturity)))/(2*kappa) )
    
    d1 = log(P_t_Ti/(Ki*P_t_T))/sd_p + sd_p/2
    d2 = d1 - sd_p
    
    # find option price
    c = P_t_Ti*norm.cdf(d1) - Ki*P_t_T*norm.cdf(d2)
    C = sum(coupons * c)
    
    return C

In [261]:
print("The price of European Option on coupon paying bond under Vasicek(with explicit formula): $", 
     round(callOptionOnCouponBS( maturity, K, F, coupons, payment_date), 5))

The price of European Option on coupon paying bond under Vasicek(with explicit formula): $ 110.86428


Tow methods (Monte Carlo Simulations) and explicit formula all return similiar results. Since Monte Carlo simulations only has a very small number of paths, the variance could be  quite large. With larger number of paths, the result of Monte Carlo simulations will converge to the result returned by explicit formula.

## Question 2

Assume the dynamics of the short-term interest rate, under the risk-neutral measure, are given by
the following SDE (CIR model):

$$dr_t =\kappa (\bar{r}− r_{t})dt + \sigma \sqrt{r_t}\ dW_t$$

with $r_0 = 5$%, $\sigma$ = 12%, $\kappa$ = 0.92, $\bar{r}$= 5.5%

**(a)** Use Monte Carlo Simulation to find at time t = 0 the price 𝑐(t, T, S) of a European Call option, with strike price of K = \$980, maturity of T = 0.5 years on a Pure Discount Bond with Face Value of \$1,000, that matures in S = 1 year:

$$P(t, T) = \ E_t\ [ exp (− \int_t^T r(s) ds)  ∗ max(P(T, S) − K, 0)]$$


** Solution: **

Initialize parameters

In [262]:
K = 980
T = 0.5 # Option maturity, t in q1
S = 1 # bond maturtiy, T in q1
F = 1000 # Face Value of zero coupon bond
r0 = 0.05 
r_mean = 0.055
sd = 0.12
kappa = 0.92 
random.seed(7)

Build function to find interest rate with CIR model

In [263]:
def CIR(r0, sd, kappa, r_mean, T, paths, steps):
    dt = T/steps 
    rt = zeros((paths, steps + 1))
    rt[:, 0] = r0
    for i in range(1, steps + 1):
        rt[:,i] = rt[:,i-1] + kappa*(r_mean - rt[:,i-1]) *dt + sd* sqrt(rt[:,i-1])* sqrt(dt)*random.normal(0, 1, paths)
    return rt

Find zero coupon bond price under CIR

In [264]:
def zeroCouponBondCIR(r0, r_mean, sd, kappa, S, T, F):
    
    paths = 1000
    steps = int(365*(S - T))
    dt = (S - T)/steps
    r = CIR(r0, sd, kappa, r_mean, (S - T), paths, steps)
    P = mean(F * exp(-sum(r, axis = 1)*dt))
    
    return P

Find European Option price  

In [265]:
def callOptionOnZeroCIR(r0, sd, kappa, r_mean, T, S, K, F, coupons, payment_date):
    
    paths = 1200
    steps = int(365*(S - T))
    dt = (S - T)/steps
    rt = CIR(r0, sd, kappa, r_mean, T, paths, steps)
    
    # Find the bond price
    P_bond = array([zeroCouponBondCIR(r, r_mean, sd, kappa, S, T, F) for r in rt[:, -1]])
    
    # Find option payoff
    payoff = maximum(P_bond - K, 0)
    P_option = mean(payoff * exp(-sum(rt, axis = 1)*dt))
    
    return P_option

In [266]:
print("The price of European Option on zero coupon bond under CIR (with Monte Carlo Simulations): $",
      round(callOptionOnZeroCIR(r0, sd, kappa, r_mean, T, S, K, F, coupons, payment_date), 5))

The price of European Option on zero coupon bond under CIR (with Monte Carlo Simulations): $ 0.41444


**(b)** Use the Implicit Finite-Difference Method to find at time t = 0 the price c(t, T, S) of a European mCall option, with strike price of K = \$980, maturity of T = 0.5 years on a Pure Discount Bond with Face Value of \$1,000, that matures in S = 1 year. The PDE is given as


$$\frac{\partial C_t}{\partial t} + \kappa(\bar{r} - r)\frac{\partial C_t}{\partial r} + \frac{1}{2}\sigma^2 r \frac{\partial ^2 C_t}{\partial r ^2} - rC_t = 0$$

with $c(T, T, S) = max(P(T, S) − K, 0)$, and $P(T, S)$ is computed explicitly.




**Solution:**

In [267]:
K = 980
T = 0.5 # Option maturity, t in q1
S = 1 # bond maturtiy, T in q1
F = 1000 # Face Value of zero coupon bond
r0 = 0.05 
r_mean = 0.055
sd = 0.12
kappa = 0.92 
random.seed(7)

Find prices of zero coupon bond explicitly

In [268]:
def ZeroCouponBondExplicitCIR(rt, r_mean, sd, kappa, T, t, F):
    
    h1 = sqrt(kappa**2 + 2*sd**2)
    h2 = (kappa + h1)/2
    h3 = (2*kappa*r_mean)/sd**2
    
    A = ((h1 * exp(h2*(T - t)))/(h2 * (exp(h1*(T - t)) - 1) + h1))**h3
    B = (exp(h1*(T - t)) - 1)/(h2 * (exp(h1*(T - t)) - 1) + h1)
    
    return F * A*exp(-B*rt)

Generate short rate grid

In [269]:
def generate_grid(sd, dr):

    return arange(0, 0.1 + dr, dr)

The parameters of $P_u$, $P_m$, $P_d$ are calculated below


$$P_u = -\frac{\Delta t}{2} \bigg[\frac{\sigma^2 r_{ij}}{\Delta r ^2} + \frac{\kappa(\bar{r} - r_{ij})}{\Delta r} \bigg]$$

$$P_m = 1 + \frac{\sigma^2 \Delta t\ r_{ij}}{\Delta r ^2} + r_{ij} \Delta t$$

$$P_u = -\frac{\Delta t}{2} \bigg[\frac{\sigma^2 r_{ij}}{\Delta r ^2} - \frac{\kappa(\bar{r} - r_{ij})}{\Delta r} \bigg]$$

In [270]:
def Generate_Ps_IFD(dt, sd, r_mean, dr, kappa, grid_r):

    Pu = -(1/2)*dt*((sd**2 * grid_r)/(dr**2) + (kappa*(r_mean - grid_r))/(dr))
    Pm = 1 + (sd**2 * dt * grid_r)/(dr**2) + grid_r*dt
    Pd = -(1/2)*dt*((sd**2 * grid_r)/(dr**2) - (kappa*(r_mean - grid_r))/(dr))
    
    return Pu, Pm, Pd

Generate matrix A for Implicit Finte Difference Method, which is the same for every period

In [271]:
def A_Imlicit(Pu, Pm, Pd, grid_size):
    
    Pd_mtx = hstack((diag(Pd[1:-1]), 
                     zeros((grid_size - 2, 2))))

    Pm_mtx = hstack((zeros((grid_size - 2, 1)), 
                     diag(Pm[1:-1]),
                     zeros((grid_size - 2, 1))))

    Pu_mtx = hstack((zeros((grid_size - 2, 2)), 
                    diag(Pu[1:-1])))
    
    A = Pu_mtx + Pm_mtx + Pd_mtx
    A = vstack((hstack((1, -1, zeros(grid_size - 2))),
                A, 
                hstack((zeros(grid_size - 2), -1, 1))))
    
    return A

Build the function that solves the PDE using Implicit Finte Difference Methods

In [272]:
def ImplicitFinteDifferenceCall(sd, r0, r_mean, kappa, T, S, K, F):
    
    # find grid parameters
    dt = 0.0001
    dr = 0.001 
    F = 1000
    
    # Generate stock grids with the input parameters
    short_rate_gird = generate_grid(sd, dr)
    grid_size =  len(short_rate_gird)
    
    # find the index that r0 exsits on the grid
    r0_idx = where(short_rate_gird == r0)

    # Generate Pu, Pm, Pd
    Pu, Pm, Pd = Generate_Ps_IFD(dt, sd, r_mean, dr, kappa, short_rate_gird)

    # Backward loop through the entire grid, solve the entire stock grid
    A_inv = linalg.inv(A_Imlicit(Pu, Pm, Pd, grid_size))
    
    # initialize matrix B
    B =  hstack(((ZeroCouponBondExplicitCIR(short_rate_gird[0], r_mean, sd, kappa, S, T, F) 
                 - ZeroCouponBondExplicitCIR(short_rate_gird[1], r_mean, sd, kappa, S, T, F)), 
                 maximum(ZeroCouponBondExplicitCIR(short_rate_gird, r_mean, sd, kappa, S, T, F) - K, 0)[1:-1], 0))

    for i in range(int(T/dt)):
       
        option_value = A_inv.dot(B)
        B =  hstack(((ZeroCouponBondExplicitCIR(short_rate_gird[0], r_mean, sd, kappa, S, T, F) 
                 - ZeroCouponBondExplicitCIR(short_rate_gird[1], r_mean, sd, kappa, S, T, F), option_value[1:-1], 0)))
    
    
    return option_value[r0_idx]

In [273]:
print("The price of European Option on zero coupon bond under CIR (with Implicit Finite Difference Method): $",
      round(ImplicitFinteDifferenceCall(sd, r0, r_mean, kappa, T, S, K, F)[0], 5))

The price of European Option on zero coupon bond under CIR (with Implicit Finite Difference Method): $ 0.39392


**(c)**Compute the price c(t, T, S) of the European Call option above using the explicit formula, and
compare it to your findings in parts (a) and (b) and comment on your findings.

**Solution:**

Build function that calculates the explicit option prices under CIR

In [274]:
def callOptionCIRExplicit(r0, sd, kappa, r_mean, T, S, K, F):
    
    P_0_T = ZeroCouponBondExplicitCIR(r0, r_mean, sd, kappa, T, 0, F)/F
    P_0_S = ZeroCouponBondExplicitCIR(r0, r_mean, sd, kappa, S, 0, F)/F

    K = K/F

    theta = sqrt(kappa**2 + 2*sd**2)
    phi = (2 * theta)/(sd**2 * (exp(theta*(T - 0)) - 1))
    si = (kappa + theta)/sd**2

    h1 = sqrt(kappa**2 + 2*sd**2)
    h2 = (kappa + h1)/2
    h3 = (2*kappa*r_mean)/sd**2

    A = ((h1 * exp(h2*(S - T)))/(h2 * (exp(h1*(S - T)) - 1) + h1))**h3
    B = (exp(h1*(S - T)) - 1)/(h2 * (exp(h1*(S - T)) - 1) + h1)

    r_star = log(A/K)/B

    C = (P_0_S* ncx2.cdf(2*r_star*(phi + si + B), 
                        (4*kappa*r_mean)/(sd**2), 
                        (2*phi**2 * r0 * exp(theta*(T-0)))/(phi + si + B))
                         - K * P_0_T * ncx2.cdf(2*r_star*(phi + si), 
                                                (4*kappa*r_mean)/(sd**2), 
                                                (2*phi**2 * r0 * exp(theta*(T-0)))/(phi + si)))
    C = F*C
    
    return C

Initialize parameters

In [275]:
K = 980
T = 0.5 # Option maturity, t in q1
S = 1 # bond maturtiy, T in q1
F = 1000 # Face Value of zero coupon bond
r0 = 0.05 
r_mean = 0.055
sd = 0.12
kappa = 0.92 

In [276]:
print("The price of European Option on zero coupon bond under CIR (with Monte Carlo Simulations): $",
      round(callOptionCIRExplicit(r0, sd, kappa, r_mean, T, S, K, F), 5))

The price of European Option on zero coupon bond under CIR (with Monte Carlo Simulations): $ 0.39406


Three methods returned very similiar results. Again, the variance of Monte Carlo Simulations are quite high. Larger number of paths are more desirable for the Monte Carlo Simulations to converge

## Question 3

Assume the dynamics of the short-term interest rate, under the risk-neutral measure, are given by
the following system of SDE (**G2++ model**):

$$dx_t = −ax_tdt + \sigma dWt^1$$
$$dy_t = −by_tdt + \eta dWt^2$$
$$r_t = x_t + y_t + \phi_t$$

$x_0$ = $y_0$ = 0, $\phi_0$ = $r_0$ = 3%, $dW_t^1dW_t^2$ = $\rho dt$, $\rho$ = 0.7, $\alpha$ = 0.1, b = 0.3, $\sigma$ = 3%, $\eta$ = 8%. Assume $\phi_t$ = const = 3% for any 𝑡 ≥ 0.


Use Monte Carlo Simulation to find at time 𝑡 = 0 the price p(t, T, S) of a European Put option,
with strike price of K = \$950, maturity of T = 0.5 years on a Pure Discount Bond with Face
value of \$1,000, that matures in S = 1 year. Compare it with the price found by the explicit
formula and comment on it.

**Solution:**

Initialize parameters

In [277]:
x0 = 0
y0 = 0
r0 = 0.03
option_maturity = 0.5
bond_maturity = 1
K = 950
random.seed(7)

Generate bivariate normal with correlation $\rho$ = 0.7

In [278]:
def bivariate_normals(paths):
    
    corr = 0.7
    Z1 = random.normal(0,1, paths)
    Z2 = random.normal(0,1, paths)
    x = Z1
    y = corr*Z1+sqrt(1-corr**2)*Z2
    
    return x,y

Generate process $r_t$

In [279]:
def G2_rt(xt, yt, alpha, beta, sigma, eta, phi, T, paths, steps):
    
    # initialize sequence
    dt = T/steps
    x = zeros((paths,steps+1))
    y = zeros((paths,steps+1))
    r = zeros((paths,steps+1))
    
    x[:,0] = xt
    y[:,0] = yt
    # generate correlated dW1,dW2
    dWt = bivariate_normals(paths*steps)
    dWt1 = dWt[0].reshape((paths,steps))
    dWt2 = dWt[1].reshape((paths,steps))
    
    for i in range(1, steps+1):

        x[:,i] = x[:,i-1] -alpha*x[:,i-1]*dt + sigma*sqrt(dt)*dWt1[:,i-1]
        y[:,i] = y[:,i-1] -beta*y[:,i-1]*dt + eta*sqrt(dt)*dWt2[:,i-1]
        
    r = x + y + phi   
    
    return r,x,y

Build the function that calculates the zero coupon bond price

In [280]:
def zeroCouponBondG2(xt, yt, t, T, F):
    phi = 0.03
    alpha = 0.1
    beta = 0.3
    sigma = 0.03
    eta = 0.08
    
    paths = 1000
    steps = 1000
    dt = (T - t)/steps

    # Genetate r paths
    r = G2_rt(xt, yt, alpha, beta, sigma, eta, phi, T, paths, steps)[0]
    
    # Find zero coupon bond price
    P = F * mean(exp(-sum(r[:,1:], axis = 1)*dt))
    
    return P

Build function that calculates the option price 

In [281]:
def putOptionOnZeroG2(xt, yt, opmat, bmat, F, K):
    # set parameters
    phi = 0.03
    alpha = 0.1
    beta = 0.3
    sigma = 0.03
    eta = 0.08
    
    #set paths and steps 
    paths = 1000
    steps = 1000
    dt = (bmat - opmat)/steps
    
    # all 3 processes from t =0 to T = option_maturity
    r_opmat,x_opmat,y_opmat = G2_rt(xt, yt, alpha, beta, sigma, eta, phi, opmat, paths, steps)
    
    # bond value in 2nd period T-S
    p_bond = array(list(map(lambda x,y: zeroCouponBondG2(x, y, opmat,bmat,F), x_opmat[:,-1], y_opmat[:,-1]))) 
    
    # Find option payoff
    payoff = maximum(K - p_bond, 0)
    P_option = mean(payoff * exp(-sum(r_opmat[:,1:], axis = 1)*dt))
    
    return P_option

In [282]:
print("The zero coupon bond price under two factor models (with explicit formula): $",
        putOptionOnZeroG2(x0, y0, opmat= option_maturity, bmat =bond_maturity , F = 1000, K =950))

The zero coupon bond price under two factor models (with explicit formula): $ 1.45944438183


Explicit Formula for two factor model

In [283]:
def zeroCouponBondExplicit(t, T, F):
    
    # set parameters
    phi = 0.03
    alpha = 0.1
    beta = 0.3
    sigma = 0.03
    eta = 0.08
    
    # break down the long fomula into 3 parts
    V1 = (sigma**2/ alpha**2)*( (T - t) + (2/alpha)*exp(-alpha*(T - t)) - (1/(2*alpha)) *exp(-2*alpha*(T - t)) - (3/(2*alpha))) 
    
    V2 = (eta**2/beta**2) *((T - t) + (2/beta)*exp(-beta * (T - t)) - (1/(2*beta)) *exp(-2 * beta * (T - t)) - (3/(2 * beta)))
        
    V3 = 2*0.7*(sigma*eta)/(alpha*beta)*((T - t) + (exp(-alpha*(T - t)) - 1)/alpha 
                                           + (exp(-beta*(T - t)) - 1)/beta -(exp(-(alpha + beta)*(T - t))-1)/(alpha + beta))
    Vt = V1 + V2 + V3
    
    P = exp(-phi*(T - t) - ((1 - exp(-alpha * (T - t)))/alpha) * x0 - ((1 - exp(-beta * (T - t))/beta)*y0) + (1/2)* Vt) * F
    
    return P

In [284]:
def europeanOptionExplicit(t, T, S, K, F):
    
    K = K/F
    xt = 0
    yt = 0
    corr = 0.7
    alpha = 0.1
    beta = 0.3
    sigma = 0.03
    eta = 0.08
    
    # break down the long formula into 3 parts
    sigma1 = sigma**2/(2* alpha**3)*(1 - exp(-alpha * (S - T)))**2 * (1 - exp(-2 *alpha *(T - t)))
    
    sigma2 = eta**2/(2*beta**3)*(1 - exp(-beta* (S - T)))**2*(1 - exp(-2*beta*(T - t)))
    
    sigma3 = (2 *0.7 *sigma *eta/(alpha*beta*(alpha + beta))
              *(1-exp(-alpha*(S - T)))*(1 - exp(-beta*(S - T)))*(1 -exp(-(alpha + beta)*(T - t))))
    
    sigma_sqr = sqrt(sigma1 + sigma2 + sigma3)

    p_t_S = zeroCouponBondExplicit(0, S, F)
    p_t_T = zeroCouponBondExplicit(0, T, F)
    
    put = (-p_t_S*norm.cdf(log(K*p_t_T/p_t_S)/sigma_sqr - sigma_sqr/2) 
           + p_t_T*K*norm.cdf(log(K*p_t_T/p_t_S)/sigma_sqr +sigma_sqr/2))
    
    return put

In [285]:
print("The zero coupon bond price under two factor models (with explicit formula): $",
     round(europeanOptionExplicit(t=0, T=0.5, S =1, K=950, F=1000), 5))

The zero coupon bond price under two factor models (with explicit formula): $ 1.86096


The European call option evaluated by these two methods returned very similiar results. the variance of Monte Carlo Simulations are quite high. Larger number of paths are more desirable for the Monte Carlo Simulations to converge. Some variance reduction techniques could be used to reduce the variance and make them converge faster